# AIMET dependencies install & build
The following group of cells installs the AIMET library for you. For more details, please see [this link](https://github.com/quic/aimet/blob/develop/packaging/google_colab_install.md).

You can clone this notebook and use it in your own project. Make sure that before running these cells, you connect to a hosted environment with a GPU accelerator. (Runtime -> Change runtime -> Hardware Accelerator(GPU))

## Installing dependencies
May prompt you.

In [ ]:
!pip3 uninstall --yes protobuf
!pip3 uninstall --yes tensorflow
!apt-get update
!apt-get install python3.6
!apt-get install python3-dev
!apt-get install python3-pip
!apt-get install liblapacke liblapacke-dev
!apt-get install wget
!pip3 install numpy==1.16.4
!apt-get install libgtest-dev build-essential cmake
!pip3 --no-cache-dir install opencv-python==4.1.0.25
!pip3 --no-cache-dir install pillow==6.2.1
!pip3 install pytorch-ignite==0.1.0
!wget -q https://github.com/Itseez/opencv/archive/3.1.0.tar.gz -O /tmp/3.1.0.tar.gz > /dev/null
!tar -C /tmp -xvf /tmp/3.1.0.tar.gz > /dev/null
%cd /tmp/opencv-3.1.0
%mkdir release
%cd release
!cmake -DCMAKE_POSITION_INDEPENDENT_CODE=ON -DBUILD_SHARED_LIBS=OFF -DCMAKE_BUILD_TYPE=release -DWITH_FFMPEG=OFF -DBUILD_TESTS=OFF -DWITH_CUDA=OFF -DBUILD_PERF_TESTS=OFF -DWITH_IPP=OFF -DENABLE_PRECOMPILED_HEADERS=OFF .. > /dev/null
!make -j16 > /dev/null
!make -j16 install > /dev/null
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
!dpkg -i cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!apt-get update
!wget http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb
!apt install ./nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb
!apt-get update
!apt install cuda-cublas-10-0 cuda-cufft-10-0 cuda-curand-10-0 cuda-cusolver-10-0
!apt-get update && apt install cuda-cusparse-10-0 libcudnn7=7.6.2.24-1+cuda10.0 libnccl2=2.4.8-1+cuda10.0  cuda-command-line-tools-10.0
!pip3 install scipy==1.1.0
!pip3 install protobuf==3.7.1
!pip3 install scikit-learn==0.19.1
!pip3 install tb-nightly==1.14.0a20190517
!pip3 install tensorboardX==1.7
!pip3 install https://download.pytorch.org/whl/cu100/torch-1.4.0%2Bcu100-cp36-cp36m-linux_x86_64.whl
!pip3 install https://download.pytorch.org/whl/cu100/torchvision-0.5.0%2Bcu100-cp36-cp36m-linux_x86_64.whl
!pip3 install --upgrade pip
!pip3 install tensorflow-gpu==1.15.0
!pip3 install future==0.17.1
!pip3 install tensorboard==1.14
!pip3 install bokeh==1.2.0
!pip3 install pandas==0.22.0
!pip3 install holoviews==1.12.7
!pip3 install --no-deps bokeh==1.2.0 hvplot==0.4.0
!pip3 install jsonschema==3.1.1
!pip3 install osqp onnx

!ln -s /usr/local/cuda-10.0 /usr/local/cuda
!apt-get update && apt-get install -y libjpeg8-dev
!ln -s /usr/lib/x86_64-linux-gnu/libjpeg.so /usr/lib

!apt install zlib1g-dev

!pip3 uninstall --yes Pillow && pip3 install Pillow-SIMD==6.0.0.post0
!pip3 uninstall --yes pytest
!pip3 install pytest

!pip3 install setuptools==41.0.1
!pip3 install keras==2.2.4

%rm -rf /usr/local/bin/python
!ln -s /usr/bin/python3 /usr/local/bin/python

After installing the dependencies, you must restart the environment before proceeding if you are working on google colab. (Runtime -> Restart Runtime) Don't run the first cell again in the restarted runtime.


## AIMET build and installation.

In [ ]:
%cd /content/
!rm -rf aimet_code
!mkdir aimet_code
%cd aimet_code
!git clone https://github.com/quic/aimet.git
%cd aimet
%mkdir -p ./ThirdParty/googletest
%pushd ./ThirdParty/googletest
!git clone https://github.com/google/googletest.git -b release-1.8.0 googletest-release-1.8.0
%popd
%cd /content/aimet_code
%mkdir build
%cd build
!cmake -DCMAKE_EXPORT_COMPILE_COMMANDS=ON ../aimet
!make -j 8
!make install

## Setting up `PYTHONPATH` and `LD_LIBRARY_PATH`

In [ ]:
import sys

sys.path.append(r'/content/aimet_code/build/staging/universal/lib/python')
sys.path.append(r'/content/aimet_code/build/staging/universal/lib/x86_64-linux-gnu')
sys.path.append(r'/usr/local/lib/python3.6/dist-packages')
sys.path.append(r'/content/aimet_code/build/artifacts')

import os

os.environ['LD_LIBRARY_PATH']+= ":/content/aimet_code/build/artifacts"

## Run unit tests
If the installation went smoothly, all tests should pass.

In [ ]:
%cd /content/aimet_code/build/
!ctest

# Import necessary python packages

In [ ]:
import os, sys
import warnings
from decimal import Decimal
import numpy as np
import matplotlib.pyplot as plt
from contextlib import contextmanager


import torchvision.transforms

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.sampler import WeightedRandomSampler
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F

from aimet_common.defs import CostMetric, CompressionScheme, GreedySelectionParameters
from aimet_torch.defs import ChannelPruningParameters, SpatialSvdParameters, ModuleCompRatioPair
from aimet_torch.compress import ModelCompressor
from aimet_torch.onnx_utils import OnnxSaver

from sklearn.metrics import f1_score
from torchvision.datasets import MNIST

# Define a model and evaluation metrics

In [ ]:
# Define test loader
transformImg = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                               torchvision.transforms.Normalize((0.1307,), (0.3081,))])


test = MNIST(root='./data', train=False, download=True, transform=transformImg)
test_loader = DataLoader(test, batch_size=1024, num_workers=4, shuffle=False, pin_memory=True)

train = MNIST(root='./data', train=True, download=True, transform=transformImg)
train_loader = DataLoader(train, batch_size=1024, num_workers=4, shuffle=False, pin_memory=True)

Set random seed for reprodubicility

In [ ]:
def set_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()

Define a model

In [ ]:
class LeNet5(torch.nn.Module):          

    def __init__(self):     
        super(LeNet5, self).__init__()
        self.convs = nn.Sequential(nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2),
                                    nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, padding=0),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2)
                                  )

        self.linears = nn.Sequential(nn.Linear(16*5*5, 120),
                                     nn.Linear(120, 84),
                                     nn.Linear(84, 10)
                                    )

    def forward(self, x):
        x = self.convs(x)
        x = x.flatten(start_dim=1)
        x = self.linears(x)

        return x

Define functions for conducting the training and testing epochs

In [ ]:
def accuracy(out, y):
    preds = out.argmax(dim=1, keepdim=True).squeeze()
    correct = preds.eq(y).sum().item()
    return correct

def train_epoch(model, opt, train_loader, criterion, device):
    model.train()
    epoch_loss = 0
    epoch_acc = 0
    n_samples = 0
    for x,y in train_loader:
        opt.zero_grad()

        x, y = x.to(device), y.to(device)
        out = model.forward(x)
        loss = criterion(out, y)

        epoch_acc += accuracy(out, y)
        epoch_loss += loss.item()
        n_samples += x.size(0)

        loss.backward()
        opt.step()

    epoch_acc = epoch_acc / n_samples
    epoch_loss = epoch_loss / n_samples

    return epoch_acc, epoch_loss

def test_epoch(model, test_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    epoch_acc = 0
    n_samples = 0
    with torch.no_grad():
        for x,y in test_loader:
            x, y = x.to(device), y.to(device)
            out = model.forward(x)
            loss = criterion(out, y)

            epoch_acc += accuracy(out, y)
            epoch_loss += loss.item()
            n_samples += x.size(0)

    epoch_acc = epoch_acc / n_samples
    epoch_loss = epoch_loss / n_samples

    return epoch_acc, epoch_loss

def display_grayscale(tensor):
    ''' show single tensor as grayscale image'''
    plt.imshow(tensor, cmap='gray')
    plt.show()

# Data processing

Define a class to apply imbalance to your datasets

In [ ]:
class Dataset_Preprocessor:
    ''' class around MNIST object for applying imbalance to its dataset '''
    def __init__(self):
        
        # Convert imgs to tensor and normalize by mean and stddev of the training set
        transformImg = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                               torchvision.transforms.Normalize((0.1307,), (0.3081,))])
        
        
        self.dataset = MNIST(root='./data', train=True, download=True, transform=transformImg)
        self.class_order = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] #MNIST-specific
        self.selection_dict = dict()
        self.shuffle()
        self.dataloader = DataLoader(self.dataset, batch_size=128, num_workers=4, shuffle=True, pin_memory=True)
    
    
    
    def shuffle(self):
        ''' shuffle dataset '''
        # generate random ordered indeces for dataset
        datapoints = self.dataset.data.shape[0]
        rand_idx = torch.randperm(datapoints)

        # shuffle data and targets in the same way
        self.dataset.data = self.dataset.data[rand_idx]
        self.dataset.targets = self.dataset.targets[rand_idx]
    
    
    
    def keep_selection(self, target, selection):
        ''' create imbalance in single class of a dataset '''

        # get indices of imgs of target number and remove selection of the indices
        target_mask = self.dataset.targets == target
        selection_idx = target_mask.nonzero()[round(len(target_mask.nonzero())*selection):]

        # make mask wich selects all data except for indices in selection
        selection_mask = np.ones(len(self.dataset.data), dtype=bool)
        selection_mask[selection_idx] = False

        # apply mask to remove the selected data
        self.dataset.data = self.dataset.data[selection_mask]
        self.dataset.targets = self.dataset.targets[selection_mask]

    
    
    def apply_imbalance(self):
        ''' create imbalance in dataset according to selection dict '''

        # throw away a part of the data for each class
        for class_number, selection in self.selection_dict.items():
            self.keep_selection(class_number, selection)
        self.dataloader = DataLoader(self.dataset, batch_size=128, num_workers=4, shuffle=True, pin_memory=True)
        self.wrs_dataloader = self.apply_weighted_random_sampler()
        
        
    def balance(self):
        self.selection_dict = dict()
        self.apply_imbalance()
        return self.selection_dict
        
        
        
    def linear_imbalance(self, rho, apply=True):
        ''' create selection dict with linear imbalance '''
        
        min_examples = 1 / rho
        
        n_steps = len(self.class_order) - 1
        linear_step = (1.0 - min_examples) / n_steps

        # interpolate the classes between the minimum and maximum linearly
        for i, data_class in enumerate(reversed(self.class_order)):
            self.selection_dict[data_class] = min_examples + (i * linear_step)
        
        if apply:
            self.apply_imbalance()

        return self.selection_dict

            
            
    def step_imbalance(self, rho, mu, apply=True):
        ''' create selection dict with step imbalance '''
        
        min_examples = 1 / rho
        
        
        n_classes = len(self.class_order)
        step_index = int(mu * n_classes)

        for i, data_class in enumerate(reversed(self.class_order)):
            if i < step_index:
                self.selection_dict[data_class] = min_examples
            else:
                self.selection_dict[data_class] = 1.0

        if apply:
            self.apply_imbalance()
            
        return self.selection_dict


                  
    def long_tailed_imbalance(self, ratio, apply=True):
        ''' create selection dict with long-tailed imbalance '''
        
        # determine mu from ratio
        mu = (1/ratio)**(1/(len(self.class_order) - 1))
        
        # set selection for each class according to long-tailed function, mu is in (0,1)
        for i, data_class in enumerate(self.class_order):
            self.selection_dict[data_class] = mu**i

        if apply:
            self.apply_imbalance()
        
        return self.selection_dict
    
    
    
    def apply_weighted_random_sampler(self):
        ''' 
        return dataloader using weighted random sampling
        modified from https://discuss.pytorch.org/t/how-to-handle-imbalanced-classes/11264 
        by user ptrblck
        '''
        
        # get sample count per class
        class_sample_count = np.unique(self.dataset.targets, return_counts=True)[1]
        
        weight = 1. / class_sample_count
        
        # define weight per sample
        samples_weight = weight[self.dataset.targets]
        samples_weight = torch.from_numpy(samples_weight)
        
        sampler = WeightedRandomSampler(samples_weight, len(samples_weight))
        
        # define weight usable for loss functions
        self.weight = torch.FloatTensor(weight / weight.sum())
        
        return DataLoader(self.dataset, batch_size=128, num_workers=4, pin_memory=True, sampler=sampler)
    
    
    
    def plot(self, title, color='r'):
        ''' plot the imbalance created by an imbalance function '''
        
        plt.bar(self.selection_dict.keys(), self.selection_dict.values(), color=color)
        plt.title(title)
        plt.show()

# Model training

Functions for training models

In [ ]:
# Focal Loss function taken from https://github.com/gokulprasadthekkel/pytorch-multi-class-focal-loss
class FocalLoss(nn.modules.loss._WeightedLoss):
    def __init__(self, weight=None, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__(weight, reduction=reduction)
        self.gamma = gamma
        self.weight = weight # weight parameter will act as the alpha parameter to balance class weights
        self.reduction = reduction
        
    def forward(self, input, target):
        ce_loss = F.cross_entropy(input, target,reduction=self.reduction, weight=self.weight)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma * ce_loss).mean()
        return focal_loss


    
def train_model(train_loader, description, loss_func="CE", weight=None, gamma=2, test=False, test_loader=test_loader, 
                log=False):
    ''' train on data loader, and return trained model '''
    
    # Determine loss function
    if loss_func == "CE":
        criterion = nn.CrossEntropyLoss(weight=weight)
    else:
        FL = FocalLoss(weight=weight, gamma=gamma)
        criterion = FL.forward
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    n_epochs = 10
    writer = SummaryWriter(f'runs/{description}')
    model = LeNet5().to(device)
    opt = optim.SGD(model.parameters(), lr=1e-2)

    # Train+test, log to tensorboard
    # It's recommended to also print all the scalar values
    for i in range(n_epochs):
        train_acc, train_loss = train_epoch(model, opt, train_loader, criterion, device)
        
        if log:
            writer.add_scalar('train/acc', train_acc, i+1)
            writer.add_scalar('train/loss', train_loss, i+1)
        
        if test:
            test_acc, test_loss = test_epoch(model, test_loader, criterion, device)
            
            if log:
                writer.add_scalar('test/acc', test_acc, i+1)
                writer.add_scalar('test/loss', test_loss, i+1)
        
    return model

# Model Compression
Using Channel Pruning and Spatial SVD

In [ ]:
# Model compressor needs an evaluation function with this specific signature

def eval_callback(model, iterations, use_cuda=True):
    model.eval()
    epoch_acc = 0
    n_samples = 0
    with torch.no_grad():
        for idx,(x,y) in enumerate(test_loader):
            if use_cuda:
                x, y = x.to('cuda:0'), y.to('cuda:0')

            out = model.forward(x)
            epoch_acc += accuracy(out, y)
            n_samples += x.size(0)

            if iterations is not None:
                if idx == iterations:
                    break
        epoch_acc = epoch_acc / n_samples

    return epoch_acc

Define model compression

In [ ]:
def compress(model, comp_ratio, method, data=None):
    """
    Compresses a trained model to a certain ratio. Uses the user specified
    compression method. There are two options: "svd" or "prun".
    If using "prun" method, user needs to provide data.
    Returns the compressed model and a list containing:
    1. baseline_model_accuracy,
    2. compressed_model_accuracy,
    3. mac_compression_ratio, 
    4. memory_compression_ratio.
    """
    input_shape = (1, 1, 28, 28)
    greedy_params = GreedySelectionParameters(target_comp_ratio=Decimal(comp_ratio))
    
    if method == "svd":      
        # Do not exclude anything in contrary to channel pruning
        modules_to_ignore = []
        auto_params = SpatialSvdParameters.AutoModeParams(greedy_params, modules_to_ignore)
        # Delete all the parameters that don't give an error in channel pruning, but do here
        spatial_svd_params = SpatialSvdParameters(mode=SpatialSvdParameters.Mode.auto,
                                                              params=auto_params)

        # This takes a bit
        comp_model, stats = ModelCompressor.compress_model(model,
                                                           input_shape=input_shape,
                                                           eval_callback=eval_callback,
                                                           eval_iterations=None,
                                                           compress_scheme=CompressionScheme.spatial_svd,
                                                           cost_metric=CostMetric.mac,
                                                           parameters=spatial_svd_params
                                                           )
        
    elif method == "prun":
        if data is None:
            print('Must provide data loader when using "prun"')
            return
        
        # Exclude first layer from pruning
        modules_to_ignore = [model.convs[0]]
        auto_params = ChannelPruningParameters.AutoModeParams(greedy_params, modules_to_ignore)
        channel_pruning_parameters = ChannelPruningParameters(mode=ChannelPruningParameters.Mode.auto,
                                                              params=auto_params,
                                                              data_loader=train_loader,
                                                              num_reconstruction_samples=1024,
                                                              allow_custom_downsample_ops=False
                                                              )

        # This takes a bit
        comp_model, stats = ModelCompressor.compress_model(model,
                                                           input_shape=input_shape,
                                                           eval_callback=eval_callback,
                                                           eval_iterations=None,
                                                           compress_scheme=CompressionScheme.channel_pruning,
                                                           cost_metric=CostMetric.mac,
                                                           parameters=channel_pruning_parameters
                                                           )
    else:
        print("Unknown method")
        return

    compare_mod = [stats.mac_compression_ratio, 
                   stats.memory_compression_ratio]
    
    return comp_model, compare_mod

In [ ]:
# COMPRESS MANUAL
# model.convs[0] = first conv layer, model.convs[3] = second conv layer
input_shape = (1, 1, 28, 28)

manual_params = SpatialSvdParameters.ManualModeParams([ModuleCompRatioPair(model.convs[0], 0.5),
                                                           ModuleCompRatioPair(model.convs[3], 0.4)])

spatial_svd_params = SpatialSvdParameters(mode=SpatialSvdParameters.Mode.manual,
                                  params=manual_params)

comp_model_svd, stats_svd = ModelCompressor.compress_model(model,
                                                   input_shape=input_shape,
                                                   eval_callback=eval_callback,
                                                   eval_iterations=None,
                                                   compress_scheme=CompressionScheme.spatial_svd,
                                                   cost_metric=CostMetric.mac,
                                                   parameters=spatial_svd_params,
                                                   )



manual_params = ChannelPruningParameters.ManualModeParams([ModuleCompRatioPair(model.convs[3], 0.4)])

channel_pruning_parameters = ChannelPruningParameters(mode=ChannelPruningParameters.Mode.manual,
                                                      params=manual_params,
                                                      data_loader=train_loader,
                                                      num_reconstruction_samples=1024,
                                                      allow_custom_downsample_ops=False
                                                      )

comp_model, stats = ModelCompressor.compress_model(model,
                                                   input_shape=input_shape,
                                                   eval_callback=eval_callback,
                                                   eval_iterations=None,
                                                   compress_scheme=CompressionScheme.channel_pruning,
                                                   cost_metric=CostMetric.mac,
                                                   parameters=channel_pruning_parameters
                                                   )

# DON'T FORGET YOU CAN CHANGE cost_metric=CostMetric.mac TO cost_metric=CostMetric.memory

Define F1-score function

In [ ]:
def model_f1_score(model, test_loader):
    ''' compute the F1 score of a model on a test dataloader '''
    model.eval()
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    y_pred = []
    y_true = []
    with torch.no_grad():
        for x,y in test_loader:
            x, y = x.to(device), y.to(device)
            out = model.forward(x)
            preds = out.argmax(dim=1, keepdim=True).squeeze()
            y_pred += preds.squeeze().tolist()
            y_true += y.squeeze().tolist()
    return f1_score(y_true, y_pred, average='weighted')

Opening a tensorboard

In [ ]:
%load_ext tensorboard
!pip3 install tensorboard-plugin-wit
%tensorboard --logdir ./runs 

### Research Question 1
Using a compression ratio of 0.5 on both channel pruning and spatial SVD.
Long-tail imbalanced is used with a ratio of 10 (most frequent class appears 10 times more often than least frequent class).

In [ ]:
# Define the dataloader
train_imbal = Dataset_Preprocessor()
train_imbal.long_tailed_imbalance(10)
train_imbal_loader = train_imbal.dataloader

model_bal = train_model(train_loader, test_loader, "Balanced")
model_imbal = train_model(train_imbal_loader, test_loader, "Imbalanced")

f1_bal = model_f1_score(model_bal, test_loader)
f1_imbal = model_f1_score(model_imbal, test_loader)

In [ ]:
comp_model_bal_svd, stats_bal = compress(model_bal, 0.5, "svd")
comp_model_imbal_svd, stats_imbal = compress(model_imbal, 0.5, "svd")

Test F1-score of compressed model.

In [ ]:
f1_bal_svd = model_f1_score(comp_model_bal_svd, test_loader)
f1_imbal_svd = model_f1_score(comp_model_imbal_svd, test_loader)

Test accuracy between balanced and imbalanced models

In [ ]:
# SVD RESULTS
# RATIO OF COMPRESSION IS 0.5
# RATIO OF IMBALANCE IS 10
print("------------------------BALANCED------------------------")
print(f"F1-score balanced model: {f1_bal}")
print(f"F1-score balanced compressed model: {f1_bal_svd}")
print("------------------------IMBALANCED------------------------")
print(f"F1-score imbalanced model: {f1_imbal}")
print(f"F1-score imbalanced compressed model: {f1_imbal_svd}")

In [ ]:
comp_model_bal_prun, stats_bal = compress(model_bal, 0.5, "prun")
comp_model_imbal_prun, stats_imbal = compress(model_imbal, 0.5, "prun")

f1_bal_prun = model_f1_score(comp_model_bal_prun, test_loader)
f1_imbal_prun = model_f1_score(comp_model_imbal_prun, test_loader)

In [ ]:
# PRUNING RESULTS
# RATIO OF COMPRESSION IS 0.5
# RATIO OF IMBALANCE IS 10
print("------------------------BALANCED------------------------")
print(f"F1-score balanced model: {f1_bal}")
print(f"F1-score balanced compressed model: {f1_bal_prun}")
print("------------------------IMBALANCED------------------------")
print(f"F1-score imbalanced model: {f1_imbal}")
print(f"F1-score imbalanced compressed model: {f1_imbal_prun}")

## Testin imbalance learning methods
HEELLL yea (temporary)

In [ ]:
# Define test loader
transformImg = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                               torchvision.transforms.Normalize((0.1307,), (0.3081,))])


test = MNIST(root='./data', train=False, download=True, transform=transformImg)
test_loader = DataLoader(test, batch_size=1024, num_workers=4, shuffle=False, pin_memory=True)

# Define train loaders
train = Dataset_Preprocessor()
train.long_tailed_imbalance(10) 
train_loader = train.dataloader
train_wrs_loader = train.wrs_dataloader
train_weight = train.weight.cuda()

print(train_weight)

In [ ]:
# normal_imb_model = train_model(train_loader, "Long-Tailed")
# wrs_CE_model = train_model(train_wrs_loader, "Long-Tailed, WRS")
# normal_WCE_model = train_model(train_loader, "Weighted CE", weight=train_weight)
# wrs_WCE_model = train_model(train_wrs_loader, "WRS, Weighted CE", weight=train_weight)
gamma_half = train_model(train_loader, "Focal Loss", loss_func="FL", weight=train_weight, gamma=0.19)
gamma_one = train_model(train_loader, "Focal Loss", loss_func="FL", weight=train_weight, gamma=0.15)

gamma_one_half = train_model(train_loader, "Focal Loss", loss_func="FL", weight=train_weight, gamma=0.25)
gamma_two = train_model(train_wrs_loader, "WRS, Focal Loss", loss_func="FL", weight=train_weight, gamma=0.21)


print("Gamma: 0.4 ;", our_f1_score(gamma_half, test_loader))
print("Gamma: 0.3 ;", our_f1_score(gamma_one, test_loader))
print("Gamma: 0.2 ;", our_f1_score(gamma_one_half, test_loader))
print("Gamma: 0.1 ;", our_f1_score(gamma_two, test_loader))

# print("Just Imbalance:", our_f1_score(normal_imb_model, test_loader))
# print("WRS, normal CE:", our_f1_score(wrs_CE_model, test_loader))
# print("Normal loader, WCE:", our_f1_score(normal_WCE_model, test_loader))
# print("WRS, WCE:", our_f1_score(wrs_WCE_model, test_loader))
# print("Normal Loader, FL:", our_f1_score(normal_FL_model, test_loader))
# print("WRS, FL:", our_f1_score(wrs_FL_model, test_loader))

# out:
# Just Imbalance: 0.9464903384582831

# WRS, normal CE: 0.9533586754650354
# Normal loader, WCE: 0.9595584417399692
# WRS, WCE: 0.897340612775242
# Normal Loader, FL: 0.9251946954174868
# WRS, FL: 0.9048888067949405
# Normal Loader, FL (gamma=3): 0.8874900486210671
# WRS, FL (gamma=1): 0.9412699364062872
# Normal Loader, FL (gamma=1): 0.9519792810834111
# WRS, FL (gamma=3): 0.8853290644996112

# Gamma: 0.5 ; 0.9520905676432927
# Gamma: 1.0 ; 0.9489965087640392
# Gamma: 1.5 ; 0.9320343237494246
# Gamma: 2.0 ; 0.8816604889961507
# Gamma: 0.4 ; 0.9521100294383325
# Gamma: 0.3 ; 0.9553254760710955
# Gamma: 0.2 ; 0.9622722563627012
# Gamma: 0.1 ; 0.9546977824804741


## Test 2
Varying the long-tailed imbalance but keeping the compress ratio constant.

In [ ]:
def imbal_ratio_tester(ratio_compr=0.5, seeds=[42]):
    ''' test how different compression ratios behave under various imbalance ratios '''
    imb_ratios = [1, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

    # test the same model for multiple seeds
    for seed in seeds:
        f1_ogs = []
        f1_svds = []
        f1_pruns = []
        set_seed(seed)
        for ratio in imb_ratios:
            # create dataloader
            train = Dataset_Preprocessor()
            train.long_tailed_imbalance(ratio)
            train_imbal_loader = train_imbal.dataloader

            # train model
            model_name = f"RQ2. Imbalance ratio: {ratio}, run: {seed}"
            model_imbal = train_model(train_imbal_loader, test_loader, model_name)

            # compress model
            comp_model_svd, stats_imbal_svd = compress(model_imbal, ratio_compr, "svd")
            comp_model_prun, stats_imbal_prun = compress(model_imbal, ratio_compr, "prun", train_imbal_loader)

            # store F1-scores
            f1_ogs.append(model_f1_score(model_imbal, test_loader))
            f1_svds.append(model_f1_score(comp_model_svd, test_loader))
            f1_pruns.append(model_f1_score(comp_model_prun, test_loader))

        # print F1-scores in a table
        print("|Imbalance\t|F1 OG Model\t|F1 SVD Model\t|F1 PRUN Model\t|")
        print("---------------------------------------------------------------------------")
        for i in range(len(ratios)):
            print(f"|{ratios[i]}\t\t|{f1_ogs[i]}\t|{f1_svds[i]}\t|{f1_pruns[i]}\t|")
            print("---------------------------------------------------------------------------")
    
    return f1_ogs, f1_svds, f1_pruns

warnings.filterwarnings("ignore")
imbal_ratio_tester(seeds=[420, 893, 666])

## Test 3

Varying the compression ratio imbalance while keeping 

In [ ]:
def comp_ratio_tester(dataloader, n_runs=1, loss_func='CE', description="", weight=None, gamma=1):
    comp_ratios = [1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]
    seeds = [31]

    for seed in seeds:
        set_seed(seed)
        f1_svds = []
        f1_pruns = []
        model = train_model(dataloader, description, weight=weight, loss_func=loss_func, gamma=gamma)
        for comp_ratio in comp_ratios:

            model_svd, stats_imbal_svd = compress(model, comp_ratio, "svd")
            model_prun, stats_imbal_prun = compress(model, comp_ratio, "prun", train_imbal_loader)

            f1_svds.append(model_f1_score(model_svd, test_loader))
            f1_pruns.append(model_f1_score(model_prun, test_loader))

        print(f"model {description}; seed: {seed}")
        print("|Imbalance\t|F1 SVD Model\t|F1 PRUN Model\t|")
        print("--------------------------------------------------------------")
        for i in range(len(comp_ratios)):
            print(f"|{comp_ratios[i]}\t|{f1_svds[i]}\t|{f1_pruns[i]}\t|")
            print("------------------------------------------------------------")

    return f1_svds, f1_pruns

warnings.filterwarnings("ignore")

# comp_ratio_tester(train_loader, description="Baseline")
comp_ratio_tester(train_wrs_loader, description="WRS")
# comp_ratio_tester(train_loader, description="WCE", weight=train_weight)
# comp_ratio_tester(train_wrs_loader, description="WRS, Weighted CE", weight=train_weight)
# comp_ratio_tester(train_loader, description="Focal Loss, gamma=1", loss_func="FL", weight=train_weight, gamma=1)
